Question #17 by Ege Özteke

In [15]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

In [16]:
df = pd.read_csv('data/municipality_bus_utilization.csv')
df.head()
df['timestamp'] = pd.to_datetime(df['timestamp'])

I first want to try working only with one municipality, say Municipality 3.

In [43]:
df3 = df.loc[df['municipality_id'] == 3]
df3.head()

,timestamp,municipality_id,usage,total_capacity
9,2017-06-04 07:59:42,3,623,1930
17,2017-06-04 08:25:42,3,770,1930
22,2017-06-04 08:59:42,3,967,1930
39,2017-06-04 09:32:46,3,1130,1930
45,2017-06-04 09:59:48,3,1263,1930


In [44]:
#I wasn't able to work with timestamp as it required the df.index to be in timestamp format, so i changed the index
df3.index = df3.timestamp

print (df3.groupby(pd.Grouper(freq='H')).max())

                              timestamp  municipality_id   usage  \
timestamp                                                          
2017-06-04 07:00:00 2017-06-04 07:59:42              3.0   623.0   
2017-06-04 08:00:00 2017-06-04 08:59:42              3.0   967.0   
2017-06-04 09:00:00 2017-06-04 09:59:48              3.0  1263.0   
2017-06-04 10:00:00 2017-06-04 10:59:48              3.0  1396.0   
2017-06-04 11:00:00 2017-06-04 11:59:44              3.0  1456.0   
...                                 ...              ...     ...   
2017-08-19 12:00:00 2017-08-19 12:30:32              3.0  1484.0   
2017-08-19 13:00:00 2017-08-19 13:30:35              3.0  1519.0   
2017-08-19 14:00:00 2017-08-19 14:30:33              3.0  1530.0   
2017-08-19 15:00:00 2017-08-19 15:29:33              3.0  1496.0   
2017-08-19 16:00:00 2017-08-19 16:30:35              3.0  1330.0   

                     total_capacity  
timestamp                            
2017-06-04 07:00:00          1930.0  
2

In [25]:
print(df3.timestamp)

9       2017-06-04 07:59:42
17      2017-06-04 08:25:42
22      2017-06-04 08:59:42
39      2017-06-04 09:32:46
45      2017-06-04 09:59:48
                ...        
13020   2017-08-19 14:30:33
13038   2017-08-19 15:03:34
13043   2017-08-19 15:29:33
13057   2017-08-19 16:03:35
13069   2017-08-19 16:30:35
Name: timestamp, Length: 1307, dtype: datetime64[ns]
